In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import networkx as nx, matplotlib.pyplot as plt, json, torch

In [3]:
from grounding.grounding_concepts import camel_case_split
objects_fn = Path('/Users/odoemoo1/projects/visual-navigation/KagNet/grounding/objects.txt')

with open(objects_fn, 'r') as f: 
    lines = f.read().split('\n')
    
objects = [line for line in lines if not line == '']
#objects_extended = [' '.join(np.unique([o] + camel_case_split(o))) for o in objects]   

In [19]:
objects

['AlarmClock',
 'Apple',
 'AppleSlice',
 'Bathtub',
 'Bed',
 'Blinds',
 'Book',
 'Bowl',
 'BowlDirty',
 'BowlFilled',
 'Box',
 'Bread',
 'BreadSliced',
 'ButterKnife',
 'Cabinet',
 'Candle',
 'CellPhone',
 'Chair',
 'Cloth',
 'CoffeeMachine',
 'Container',
 'ContainerFull',
 'CounterTop',
 'CreditCard',
 'Cup',
 'Dirt',
 'Egg',
 'EggFried',
 'EggShell',
 'Fork',
 'Fridge',
 'GarbageCan',
 'HousePlant',
 'KeyChain',
 'Knife',
 'Lamp',
 'Laptop',
 'Lettuce',
 'LettuceSliced',
 'LightSwitch',
 'Microwave',
 'Mirror',
 'Mug',
 'MugFilled',
 'Newspaper',
 'Omelette',
 'Painting',
 'PaintingHanger',
 'Pan',
 'Pen',
 'Pencil',
 'Plate',
 'Plunger',
 'Pot',
 'Potato',
 'PotatoSliced',
 'RemoteControl',
 'Sandwich',
 'ScrubBrush',
 'ShowerDoor',
 'Sink',
 'SoapBar',
 'SoapBottle',
 'Spoon',
 'SportsEquipment',
 'SprayBottle',
 'Statue',
 'StoveBurner',
 'StoveKnob',
 'TableTop',
 'Television',
 'TissueBox',
 'Toaster',
 'Toilet',
 'ToiletPaper',
 'Tomato',
 'TomatoSliced',
 'Towel',
 'TowelHold

In [4]:
with open('./conceptnet/concept2id.json') as f:
    concept2id = json.load(f)

In [5]:
cpnet = nx.read_gpickle('./conceptnet/cpnet.graph')
cpnet_simple = nx.Graph()
for u, v, data in cpnet.edges(data=True):
    w = data['weight'] if 'weight' in data else 1.0
    if cpnet_simple.has_edge(u, v):
        cpnet_simple[u][v]['weight'] += w
    else:
        cpnet_simple.add_edge(u, v, weight=w)


In [7]:
objs_in_concept = [oo for oo in objects if oo.lower() in concept2id.keys()]
objs_not_in_concept = [oo for oo in objects if not oo.lower() in concept2id.keys()]
objs_not_in_concept_extended = [' '.join(np.unique([o] + camel_case_split(o))) for o in objs_not_in_concept]   
all_objs = objs_in_concept + objs_not_in_concept_extended

In [8]:
nodes = []
edges = []
output_graph_text = ""
graphs = {}
out_graph_fn = Path('./datasets/ai2thor/ego_graphs.graph')

for obj in all_objs:
    sub_graphs = []
    
    for oo in obj.split(' '):
        try:
            ego_graph = nx.ego_graph(cpnet_simple, concept2id[oo.lower()])
            #plt.figure()
            #nx.draw(ego_graph); plt.show()
            #print(f'\t{oo.lower()} nodes: {ego_graph.number_of_nodes()}, edges: {ego_graph.number_of_edges()}')
            sub_graphs.append(ego_graph)
        except:
            pass
        
    if len(sub_graphs):
        combined_graph = nx.compose_all(sub_graphs)
        print(f'{obj}, \tnodes: {combined_graph.number_of_nodes()}, \tedges: {combined_graph.number_of_edges()}')

        edges.append(combined_graph.number_of_edges())
        nodes.append(combined_graph.number_of_nodes())
        
        _combined_graph = nx.convert_node_labels_to_integers(combined_graph, label_attribute='cid')
        graph_str = json.dumps(nx.node_link_data(_combined_graph))
        
        output_graph_text += graph_str + "\n"


with open(out_graph_fn, 'w') as fw:
    fw.write(output_graph_text)


AlarmClock, 	nodes: 2, 	edges: 1
Apple, 	nodes: 500, 	edges: 1214
Bathtub, 	nodes: 59, 	edges: 190
Bed, 	nodes: 695, 	edges: 2550
Blinds, 	nodes: 10, 	edges: 16
Book, 	nodes: 1262, 	edges: 3767
Bowl, 	nodes: 260, 	edges: 658
Box, 	nodes: 676, 	edges: 2441
Bread, 	nodes: 598, 	edges: 1697
ButterKnife, 	nodes: 2, 	edges: 1
Cabinet, 	nodes: 387, 	edges: 929
Candle, 	nodes: 174, 	edges: 397
CellPhone, 	nodes: 19, 	edges: 29
Chair, 	nodes: 316, 	edges: 1052
Cloth, 	nodes: 448, 	edges: 1158
Container, 	nodes: 392, 	edges: 1381
CounterTop, 	nodes: 15, 	edges: 19
Cup, 	nodes: 247, 	edges: 705
Dirt, 	nodes: 171, 	edges: 616
Egg, 	nodes: 715, 	edges: 1813
EggShell, 	nodes: 12, 	edges: 13
Fork, 	nodes: 163, 	edges: 449
Fridge, 	nodes: 115, 	edges: 273
HousePlant, 	nodes: 30, 	edges: 44
KeyChain, 	nodes: 7, 	edges: 8
Knife, 	nodes: 336, 	edges: 829
Lamp, 	nodes: 185, 	edges: 456
Laptop, 	nodes: 39, 	edges: 86
Lettuce, 	nodes: 73, 	edges: 181
LightSwitch, 	nodes: 2, 	edges: 1
Microwave, 	nodes: 76,

In [13]:
import dgl
def read_file(fname, f_type='txt'):
    assert Path(fname).is_file(), f'{fname} does not exist.'

    with open(fname, 'r') as f:
        if f_type == 'txt':
            out = [line.strip() for line in f.readlines()]
        elif f_type == 'json':
            out = json.load(f)
        
    return out

def load_graphs(objects_fname, graph_nxg_file):
    """
    Pre-generated graphs with relationships between each object. 
    Generated via KagNet repo.
    The number of objects is fixed, although the number of nodes differ.
    """
    objects = read_file(objects_fname)
    nxgs = read_file(graph_nxg_file)
    dgs = {}
    nxgs = nxgs[:len(objects)]

    for ii, nxg_str in enumerate(nxgs):
        nxg = nx.node_link_graph(json.loads(nxg_str))
        dg = dgl.DGLGraph()
        dg.from_networkx(nxg)
         #adding 1, as 0 stands for placeholder / dummy concept
        cids = [nxg.nodes[n_id]['cid'] + 1 for n_id in range(len(dg))]
        dg.ndata.update({'conceptnet_ids': torch.LongTensor(cids)})
        dgs[objects[ii]] = dg
    
    return dgs


Using backend: pytorch


In [14]:
objects_fn = Path('/Users/odoemoo1/projects/visual-navigation/KagNet/grounding/objects.txt')
dgs = load_graphs(objects_fname=objects_fn, graph_nxg_file=out_graph_fn)

In [15]:
dgs

{'AlarmClock': DGLGraph(num_nodes=2, num_edges=2,
          ndata_schemes={'conceptnet_ids': Scheme(shape=(), dtype=torch.int64)}
          edata_schemes={}),
 'Apple': DGLGraph(num_nodes=500, num_edges=2428,
          ndata_schemes={'conceptnet_ids': Scheme(shape=(), dtype=torch.int64)}
          edata_schemes={}),
 'AppleSlice': DGLGraph(num_nodes=59, num_edges=380,
          ndata_schemes={'conceptnet_ids': Scheme(shape=(), dtype=torch.int64)}
          edata_schemes={}),
 'Bathtub': DGLGraph(num_nodes=695, num_edges=5100,
          ndata_schemes={'conceptnet_ids': Scheme(shape=(), dtype=torch.int64)}
          edata_schemes={}),
 'Bed': DGLGraph(num_nodes=10, num_edges=32,
          ndata_schemes={'conceptnet_ids': Scheme(shape=(), dtype=torch.int64)}
          edata_schemes={}),
 'Blinds': DGLGraph(num_nodes=1262, num_edges=7534,
          ndata_schemes={'conceptnet_ids': Scheme(shape=(), dtype=torch.int64)}
          edata_schemes={}),
 'Book': DGLGraph(num_nodes=260, num_edges=1

In [ ]:
def camel_case_split(str): 
    return re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', str)

def get_conceptnet_graph(conceptnet_graph_fname):
    cpnet = nx.read_gpickle(conceptnet_graph_fname)
    cpnet_simple = nx.Graph()

    for u, v, data in cpnet.edges(data=True):
        w = data['weight'] if 'weight' in data else 1.0

        if cpnet_simple.has_edge(u, v):
            cpnet_simple[u][v]['weight'] += w
        else:
            cpnet_simple.add_edge(u, v, weight=w)

    del cpnet
    return cpnet_simple

def generate_graphs(objects_fname, conceptnet_graph_fname, conceptnet_id_fname):
    """
    Generate graphs for each object, where each object is at the center of the graph. 
    The graph consists of all the neighbors; also refered to as ego-subgraph.
    """
    concept2id = read_file(conceptnet_id_fname,f_type='json')
    conceptnet_graph = get_conceptnet_graph(conceptnet_graph_fname)
    
    objects = read_file(objects_fname)
    objs_in_concept = [oo for oo in objects if oo.lower() in concept2id.keys()]
    objs_not_in_concept = [oo for oo in objects if not oo.lower() in concept2id.keys()]
    objs_not_in_concept_extended = [' '.join(np.unique([o] + camel_case_split(o))) for o in objs_not_in_concept]   
    all_objs = objs_in_concept + objs_not_in_concept_extended
    obj_map = {for oo in all_objects}
    graphs = {}

    for obj in all_objs:
        sub_graphs = []
        for oo in obj.split(' '):
            try:
                ego_graph = nx.ego_graph(conceptnet_graph, concept2id[oo.lower()])
                sub_graphs.append(ego_graph)
            except:
                pass
            
        if len(sub_graphs):
            combined_graph = nx.compose_all(sub_graphs)
            combined_graph = nx.convert_node_labels_to_integers(combined_graph, label_attribute='cid')
            dg = dgl.DGLGraph()
            dg.from_networkx(combined_graph)
            combined_graph_json = nx.node_link_data(combined_graph)
            cids = [combined_graph_json.nodes[n_id]['cid'] + 1 for n_id in range(len(dg))] #adding 1, as 0 stands for placeholder / dummy concept
            dg.ndata.update({'conceptnet_ids': torch.LongTensor(cids)})
            graphs[max(obj.split(' '))] = dg

    return graphs





In [16]:
!git status

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   embeddings/OpenKE/openke/config/Trainer.py
	modified:   embeddings/OpenKE/train_transe_conceptnet.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	conceptnet/concept2id.json
	conceptnet/id2concept.json
	conceptnet/id2relation.json
	conceptnet/relation2id.json
	semantic_ego_subgraph_generation.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [17]:
!add 

apl	https://gitlab.jhuapl.edu/odoemoo1/knowledge-aware-graph-net.git (fetch)
apl	https://gitlab.jhuapl.edu/odoemoo1/knowledge-aware-graph-net.git (push)
fork	https://github.com/kachiO/KagNet.git (fetch)
fork	https://github.com/kachiO/KagNet.git (push)
